In [ ]:
# ========================================================================
# ===== USER INPUTS ======================================================
# ========================================================================
gdb = r"D:\GIS\REEF_ISLAND_RESEARCH.gdb"

# Feature-class wildcards
GE_WILDCARD = "*_GE_poly_Project"      # e.g. Kodingarengkeke_GE_poly_Project
CS_WILDCARD = "Csat_update_*_poly"     # e.g. Csat_update_kodkeke_poly

# IoU table (created in Part 1, used again in Parts 2 & 3)
out_table_name = "IoU_all_sites_nearestDate"
MAX_DAYS_DIFF  = 14   # nearest-date matching threshold (days)
ONE_TO_ONE     = True # each CoastSat polygon used at most once per Site

# Time-series plots (Part 2)
out_dir_timeseries = r"D:\GIS\plots\IoU_timeseries14day"

# Boxplot with single-time UAV values (Part 3)
out_png_box = r"D:\GIS\plots\IoU_timeseries14day\IoU_boxplot_slide2_style.png"

# Single-time IoU (%) from manual UAV–CoastSat maps
single_time_pct = {
    "Podang Podang Caddi": 95.0,
    "Podang Podang Lompo": 79.0,
    "Kodingareng Keke":     59.0,
}

# Site color palette
SITE_COLORS = {
    "Kodingareng Keke":      "#1f77b4",
    "Podang Podang Caddi":   "#2ca02c",
    "Podang Podang Lompo":   "#9467bd",
}

# ========================================================================
# ===== IMPORTS & GLOBAL SETTINGS ========================================
# ========================================================================
import arcpy, os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib as mpl

arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True

mpl.rcParams.update({
    "figure.dpi": 300,
    "axes.labelsize": 16,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "legend.fontsize": 12
})
# Use constrained_layout for complex multi-panel figure
plt.rcParams['figure.constrained_layout.use'] = True

# ========================================================================
# ===== HELPER FUNCTIONS =================================================
# ========================================================================
def parse_date(val):
    """Return datetime.date from many formats or None."""
    if val is None:
        return None
    if hasattr(val, "date"):
        try:
            return val.date()
        except Exception:
            pass
    s = str(val).strip()
    if not s:
        return None
    fmts = [
        "%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M",
        "%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y",
        "%d-%m-%Y", "%m-%d-%Y", "%Y/%m/%d",
    ]
    for f in fmts:
        try:
            return datetime.strptime(s, f).date()
        except Exception:
            pass
    try:
        return datetime.fromisoformat(s[:10]).date()
    except Exception:
        return None

def find_field(fc, candidates):
    flds = {f.name.lower(): f.name for f in arcpy.ListFields(fc)}
    for c in candidates:
        if c.lower() in flds:
            return flds[c.lower()]
    return None

def slug(s):
    """Lowercase alnum slug (remove spaces/punct)."""
    return "".join(ch for ch in s.lower() if ch.isalnum())

# Site mapping to unify synonyms/shortcodes → canonical label
SITE_CANON = {
    "kodingarengkeke": "Kodingareng Keke",
    "podangpodangcaddi": "Podang Podang Caddi",
    "podangpodanglompo": "Podang Podang Lompo",
    "kodkeke": "Kodingareng Keke",
    "podcaddi": "Podang Podang Caddi",
    "podlompo": "Podang Podang Lompo",
    "kodingarengkek": "Kodingareng Keke",
    "kodingarengke": "Kodingareng Keke",
    "podangcaddi": "Podang Podang Caddi",
    "podanglompo": "Podang Podang Lompo",
}

def canon_site(raw, fcname):
    """Prefer Site field; else derive from FC name; then normalize via mapping."""
    if raw is not None:
        s = str(raw).strip()
        if s:
            key = slug(s)
            return SITE_CANON.get(key, s)
    base = os.path.basename(fcname)
    base = base.replace("_GE_poly_Project", "").replace("Csat_update_", "").replace("_poly", "")
    key = slug(base)
    return SITE_CANON.get(key, base.replace("_", " ").title())

def iou_from_geoms(geom_ge, geom_cs):
    """
    Compute IoU and over/under-estimation areas.

    Returns:
        iou          (0–1)
        a_inter_m2   (intersection area)
        a_union_m2   (union area)
        a_ge_m2      (GE reference area)
        a_cs_m2      (CoastSat area)
        over_m2      (CoastSat outside GE)
        under_m2     (GE outside CoastSat)
    """
    if geom_ge is None or geom_cs is None:
        return 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

    inter = geom_ge.intersect(geom_cs, 4)

    try:
        a_inter = float(getattr(inter, "area", 0.0))
    except Exception:
        a_inter = 0.0

    try:
        a_ge = float(geom_ge.area)
    except Exception:
        a_ge = 0.0
    try:
        a_cs = float(geom_cs.area)
    except Exception:
        a_cs = 0.0

    a_union = max(a_ge + a_cs - a_inter, 0.0)
    iou = (a_inter / a_union) if a_union > 0 else 0.0

    over_m2  = max(a_cs - a_inter, 0.0)  # CoastSat-only
    under_m2 = max(a_ge - a_inter, 0.0)  # GE-only

    return iou, a_inter, a_union, a_ge, a_cs, over_m2, under_m2

def set_ylim_with_padding(ax, valsA, valsB=None, pad=8):
    """Helper for boxplot y-limits with a bit of headroom."""
    tops = list(valsA)
    if valsB is not None:
        tops += [v for v in valsB if np.isfinite(v)]
    ymax = max(tops) if tops else 100.0
    ax.set_ylim(0, min(100, ymax + pad))

# ========================================================================
# ===== PART 1: BUILD IoU TABLE (NEAREST-DATE MATCHING) ==================
# ========================================================================

ge_fcs = arcpy.ListFeatureClasses(GE_WILDCARD) or []
cs_fcs = arcpy.ListFeatureClasses(CS_WILDCARD) or []

if not ge_fcs:
    raise RuntimeError(f"No GE feature classes found with wildcard {GE_WILDCARD}")
if not cs_fcs:
    raise RuntimeError(f"No CoastSat feature classes found with wildcard {CS_WILDCARD}")

print("GE layers:", ge_fcs)
print("CoastSat layers:", cs_fcs)

# read features: bucket by Site only
date_candidates = ["Date_std", "Date", "date"]
GE_by_site = {}
CS_by_site = {}

def add_site_record(store, site, rec):
    store.setdefault(site, []).append(rec)

def sweep_fc(fc, is_ge):
    dfld = find_field(fc, date_candidates)
    sfld = find_field(fc, ["Site", "site"])
    if not dfld:
        raise RuntimeError(f"No date field in {fc}. Add one of: {date_candidates}")
    fields = ["OID@", "SHAPE@", dfld] + ([sfld] if sfld else [])

    bad_dates = 0
    with arcpy.da.SearchCursor(fc, fields) as cur:
        for row in cur:
            d = parse_date(row[2])
            if d is None:
                bad_dates += 1
                continue
            site = canon_site(row[3] if sfld else None, fc)
            rec = {"date": d, "geom": row[1], "oid": row[0], "site": site, "fc": fc}
            if is_ge:
                add_site_record(GE_by_site, site, rec)
            else:
                add_site_record(CS_by_site, site, rec)
    if bad_dates:
        print(f"WARNING: {fc} had {bad_dates} rows with unparsed dates (skipped).")

for fc in ge_fcs:
    sweep_fc(fc, True)
for fc in cs_fcs:
    sweep_fc(fc, False)

common_sites = sorted(set(GE_by_site.keys()) & set(CS_by_site.keys()))
print("Sites with both GE and CoastSat:", common_sites)

# match by nearest date (within MAX_DAYS_DIFF)
matches = []

for site in common_sites:
    ge_list = sorted(GE_by_site[site], key=lambda r: r["date"])
    cs_list = sorted(CS_by_site[site], key=lambda r: r["date"])

    used_cs = set()  # which CS OIDs already paired (if ONE_TO_ONE)

    for ge_rec in ge_list:
        best_cs = None
        best_dt = None  # abs time diff in days

        for cs_rec in cs_list:
            if ONE_TO_ONE and cs_rec["oid"] in used_cs:
                continue
            dt_days = abs((cs_rec["date"] - ge_rec["date"]).days)
            if best_dt is None or dt_days < best_dt:
                best_dt = dt_days
                best_cs = cs_rec

        if best_cs is None or best_dt is None or best_dt > MAX_DAYS_DIFF:
            continue

        if ONE_TO_ONE:
            used_cs.add(best_cs["oid"])

        (iou, a_inter, a_union, a_ge, a_cs, over_m2, under_m2) = iou_from_geoms(
            ge_rec["geom"], best_cs["geom"]
        )

        matches.append({
            "Site": site,
            "Year": ge_rec["date"].year,
            "Month": ge_rec["date"].month,
            "Date_GE": ge_rec["date"],
            "Date_CS": best_cs["date"],
            "Days_Diff": int(best_dt),
            "OID_GE": ge_rec["oid"],
            "OID_CS": best_cs["oid"],
            "Area_GE_m2": a_ge,
            "Area_CS_m2": a_cs,
            "Area_Inter_m2": a_inter,
            "Area_Union_m2": a_union,
            "IoU": iou,
            "IoU_pct": iou * 100.0,
            "Over_m2": over_m2,
            "Under_m2": under_m2,
            "Over_ha": over_m2 / 10000.0,
            "Under_ha": under_m2 / 10000.0,
        })

print(f"Matched pairs (nearest date within {MAX_DAYS_DIFF} days): {len(matches)}")

# write output table
out_table = os.path.join(gdb, out_table_name)
if arcpy.Exists(out_table):
    arcpy.management.Delete(out_table)
arcpy.management.CreateTable(gdb, out_table_name)

schema = [
    ("Site", "TEXT", 80),
    ("Year", "LONG", None),
    ("Month", "LONG", None),
    ("Date_GE", "DATE", None),
    ("Date_CS", "DATE", None),
    ("Days_Diff", "LONG", None),
    ("OID_GE", "LONG", None),
    ("OID_CS", "LONG", None),
    ("Area_GE_m2", "DOUBLE", None),
    ("Area_CS_m2", "DOUBLE", None),
    ("Area_Inter_m2", "DOUBLE", None),
    ("Area_Union_m2", "DOUBLE", None),
    ("IoU", "DOUBLE", None),
    ("IoU_pct", "DOUBLE", None),
    ("Over_m2", "DOUBLE", None),
    ("Under_m2", "DOUBLE", None),
    ("Over_ha", "DOUBLE", None),
    ("Under_ha", "DOUBLE", None),
]

for nm, tp, ln in schema:
    if ln is None:
        arcpy.management.AddField(out_table, nm, tp)
    else:
        arcpy.management.AddField(out_table, nm, tp, field_length=ln)

fields = [s[0] for s in schema]
with arcpy.da.InsertCursor(out_table, fields) as icur:
    for r in matches:
        icur.insertRow([r[f] for f in fields])

print(f"Done. Wrote {len(matches)} matched IoU rows to: {out_table}")

# ========================================================================
# ===== PART 2 & 3 COMBINED: BOX + 3× TIME-SERIES =======================
# ========================================================================

os.makedirs(out_dir_timeseries, exist_ok=True)

table_path = os.path.join(gdb, out_table_name)
if not arcpy.Exists(table_path):
    raise RuntimeError(f"Table not found: {out_table_name}")

# --- Single-time UAV stats from your accuracy map ----------------------
# IoU in percent, Over/Under in hectares
single_site_stats = {
    "Kodingareng Keke": {
        "IoU_pct": 59.0,
        "Over_ha": 0.85,
        "Under_ha": 0.01,
    },
    "Podang Podang Caddi": {
        "IoU_pct": 95.0,
        "Over_ha": 0.12,
        "Under_ha": 0.10,
    },
    "Podang Podang Lompo": {
        "IoU_pct": 79.0,
        "Over_ha": 1.27,
        "Under_ha": 0.02,
    },
}

# UAV dates (single-time) you confirmed
uav_dates = {
    "Kodingareng Keke":      pd.to_datetime("2016-07-24"),
    "Podang Podang Caddi":   pd.to_datetime("2017-07-19"),
    "Podang Podang Lompo":   pd.to_datetime("2017-07-19"),
}

# helper from earlier – now mainly used for other plots, can stay here
def set_xlim_with_padding(ax, valsA, valsB=None, pad=8):
    xs = [v for v in valsA if np.isfinite(v)]
    if valsB is not None:
        xs += [v for v in valsB if np.isfinite(v)]
    if not xs:
        ax.set_xlim(0, 100)
        return
    xmax = max(xs)
    ax.set_xlim(0, min(100, xmax + pad))

# -----------------------------------------------------------------------
# 1. Read IoU table for both time series AND boxplot
# -----------------------------------------------------------------------

# For time series (multi-time areas)
fields_ts = ["Site", "Date_CS", "Area_Inter_m2", "Over_ha", "Under_ha"]
rows_ts = []
with arcpy.da.SearchCursor(table_path, fields_ts) as cur:
    for site, date_cs, a_inter_m2, over_ha, under_ha in cur:
        if date_cs is None or site is None:
            continue
        inter_ha = (a_inter_m2 or 0.0) / 10000.0
        rows_ts.append({
            "Site": site,
            "Date_CS": date_cs,
            "Over_ha": over_ha or 0.0,
            "Under_ha": under_ha or 0.0,
            "Inter_ha": inter_ha,
        })

df_ts = pd.DataFrame(rows_ts)
if df_ts.empty:
    raise RuntimeError("IoU table has no usable rows for time series.")
df_ts["Date_CS"] = pd.to_datetime(df_ts["Date_CS"])

# For boxplot (IoU % distribution)
fields_box = ["Site", "IoU_pct"]
rows_box = []
with arcpy.da.SearchCursor(table_path, fields_box) as cur:
    for site, iou_pct in cur:
        if site is None or iou_pct is None:
            continue
        rows_box.append({"Site": site, "IoU_pct": float(iou_pct)})

df_box = pd.DataFrame(rows_box)
if df_box.empty:
    raise RuntimeError("No valid IoU records found for boxplot.")

# -----------------------------------------------------------------------
# 2. Define consistent site order for both box and time series
# -----------------------------------------------------------------------
SITE_ORDER = [
    "Podang Podang Lompo",   # top
    "Podang Podang Caddi",   # middle
    "Kodingareng Keke",      # bottom
]

valid_sites = [s for s in SITE_ORDER if s in df_box["Site"].unique()]
if not valid_sites:
    raise RuntimeError("No valid sites found in IoU table for plotting.")

site_labels_short = {
    "Kodingareng Keke":      "K. Keke",
    "Podang Podang Caddi":   "P. Caddi",
    "Podang Podang Lompo":   "P. Lompo",
}

# -----------------------------------------------------------------------
# 3. IoU summary per site (for labels & limits)
# -----------------------------------------------------------------------
summary = (
    df_box.groupby("Site")["IoU_pct"]
    .agg(["count", "mean", "std", "min", "max"])
    .reset_index()
)

means = []
ns = []
single_vals = []
site_min_vals = {}

for s in valid_sites:
    vals = df_box.loc[df_box["Site"] == s, "IoU_pct"].values
    if len(vals) == 0:
        continue
    site_min_vals[s] = float(np.min(vals))
    means.append(float(np.mean(vals)))
    ns.append(int(len(vals)))
    single_vals.append(single_site_stats.get(s, {}).get("IoU_pct", np.nan))

# -----------------------------------------------------------------------
# 4. Pre-compute global y-range for time-series panels (aligned scales)
# -----------------------------------------------------------------------
global_ymax = 0.0
for s in valid_sites:
    sub = df_ts[df_ts["Site"] == s]
    if sub.empty:
        continue
    local_max = max(
        sub["Over_ha"].max(),
        sub["Under_ha"].max(),
        sub["Inter_ha"].max(),
    )
    stats = single_site_stats.get(s)
    if stats is not None:
        r = stats["IoU_pct"] / 100.0
        O = stats["Over_ha"]
        U = stats["Under_ha"]
        I = (r * (O + U)) / (1.0 - r)
        local_max = max(local_max, O, U, I)
    global_ymax = max(global_ymax, local_max)

if global_ymax <= 0:
    global_ymax = 1.0

global_ylim = float(np.ceil(global_ymax * 1.1 * 2.0) / 2.0)  # to nearest 0.5
yticks = np.arange(0.0, global_ylim + 0.001, 1.0)
# -----------------------------------------------------------------------
# 5. Filter data to valid sites
# -----------------------------------------------------------------------
df_box = df_box[df_box["Site"].isin(valid_sites)].copy()
df_ts  = df_ts[df_ts["Site"].isin(valid_sites)].copy()

# -----------------------------------------------------------------------
# 6. IoU summary for labels and global y-limits (time series)
# -----------------------------------------------------------------------
summary = (
    df_box.groupby("Site")["IoU_pct"]
    .agg(["count", "mean", "min", "max"])
    .reindex(valid_sites)
)

site_min_vals = summary["min"].to_dict()
site_means    = summary["mean"].to_dict()
site_ns       = summary["count"].to_dict()

single_vals = {s: single_site_stats.get(s, {}).get("IoU_pct", np.nan)
               for s in valid_sites}

# Global y-range for time series
global_ymax = 0.0
for s in valid_sites:
    sub = df_ts[df_ts["Site"] == s]
    if not sub.empty:
        local_max = max(sub["Over_ha"].max(),
                        sub["Under_ha"].max(),
                        sub["Inter_ha"].max())
    else:
        local_max = 0.0

    stats = single_site_stats.get(s)
    if stats is not None:
        r = stats["IoU_pct"] / 100.0
        O = stats["Over_ha"]
        U = stats["Under_ha"]
        I = (r * (O + U)) / (1.0 - r) if r < 1.0 else 0.0
        local_max = max(local_max, O, U, I)

    global_ymax = max(global_ymax, local_max)

if global_ymax <= 0:
    global_ymax = 1.0

global_ylim = float(np.ceil(global_ymax * 1.1 * 2.0) / 2.0)
yticks = np.arange(0.0, global_ylim + 0.001, 1.0)

# -----------------------------------------------------------------------
# 7. Figure layout (constrained_layout)
# -----------------------------------------------------------------------
fig = plt.figure(figsize=(14, 8.5), dpi=300, constrained_layout=True)

gs = fig.add_gridspec(
    nrows=len(valid_sites),
    ncols=2,
    width_ratios=[1.3, 2.0],
)

# Left: boxplot spanning all site rows
ax_box = fig.add_subplot(gs[:, 0])

# Right: one time-series axis per site (top→bottom, shared x)
axes_ts = []
for i in range(len(valid_sites)):
    if i == 0:
        ax_ts = fig.add_subplot(gs[i, 1])
    else:
        ax_ts = fig.add_subplot(gs[i, 1], sharex=axes_ts[0])
    axes_ts.append(ax_ts)

# -----------------------------------------------------------------------
# 8. Vertical boxplot (IoU vs Site) with side-by-side star + labels + A
# -----------------------------------------------------------------------
all_vals_for_ylim = list(df_box["IoU_pct"].values)

for i, s in enumerate(valid_sites):
    vals = df_box.loc[df_box["Site"] == s, "IoU_pct"].values
    if len(vals) == 0:
        continue

    m  = site_means[s]
    sv = single_vals[s]
    all_vals_for_ylim.append(m)
    if np.isfinite(sv):
        all_vals_for_ylim.append(sv)

    # X positions: box and star side-by-side around site index
    pos_center = float(i)
    pos_box    = pos_center - 0.15   # left
    pos_star   = pos_center + 0.15   # right

    # Boxplot at pos_box
    bp = ax_box.boxplot(
        vals,
        positions=[pos_box],
        vert=True,
        widths=0.25,
        patch_artist=True,
        showmeans=False,
        manage_ticks=False,
    )

    color = SITE_COLORS.get(s, "#333333")
    for box in bp["boxes"]:
        box.set(facecolor="white", edgecolor=color, linewidth=1.8)
    for w in bp["whiskers"]:
        w.set(color=color, linewidth=1.5)
    for cap in bp["caps"]:
        cap.set(color=color, linewidth=1.5)
    for med in bp["medians"]:
        med.set(color=color, linewidth=2.0)

    # Single-time UAV IoU star at pos_star
    if np.isfinite(sv):
        ax_box.plot(
            pos_star,
            sv,
            marker="*",
            markersize=13,
            linestyle="None",
            color=color,
        )

    # Labels directly under each symbol
    site_min = site_min_vals.get(s, m)
    # μ / n under the box
    y_label_box = max(0.0, site_min - 8.0)
    ax_box.text(
        pos_box,
        y_label_box,
        f"\u03bc={m:.0f}%\n"
        f"n={site_ns[s]}",
        ha="center",
        va="top",
        fontsize=11,
    )

    # single-time % under the star
    if np.isfinite(sv):
        y_label_star = max(0.0, sv - 8.0)
        ax_box.text(
            pos_star,
            y_label_star,
            f"{sv:.0f}%",
            ha="center",
            va="top",
            fontsize=11,
        )

# Axis formatting for boxplot
if all_vals_for_ylim:
    y_max = min(100.0, max(all_vals_for_ylim) + 8.0)
else:
    y_max = 100.0

ax_box.set_ylim(0.0, y_max)
ax_box.yaxis.set_major_formatter(FuncFormatter(lambda y, pos: f"{y:.0f}"))
ax_box.set_ylabel("Intersection over Union (IoU) [%]", rotation=90, labelpad=10)

xticks_box = np.arange(len(valid_sites))
ax_box.set_xticks(xticks_box)
ax_box.set_xticklabels([site_labels_short.get(s, s) for s in valid_sites])
ax_box.set_xlabel("Site")

# Panel letter A
ax_box.text(
    0.02, 0.98, "A",
    transform=ax_box.transAxes,
    ha="left", va="top",
    fontsize=14, fontweight="bold",
)

# -----------------------------------------------------------------------
# 9. Time-series panels (B–D)
# -----------------------------------------------------------------------
legend_handles_all = [
    Patch(facecolor="white", edgecolor="black", linewidth=1.2,
          label="Multiple time – GEP"),
    Line2D([0], [0], marker="*", linestyle="None", markersize=10,
           color="dimgray", label="Single time – UAV"),
    Line2D([0], [0], color="dimgray", linestyle="-", marker="o",
           markersize=4, linewidth=1.4, label="Over"),
    Line2D([0], [0], color="dimgray", linestyle="--", marker="s",
           markersize=4, linewidth=1.4, label="Under"),
    Line2D([0], [0], color="dimgray", linestyle=":", marker="^",
           markersize=4, linewidth=1.4, label="Intersect"),
]

for i, (s, ax) in enumerate(zip(valid_sites, axes_ts)):
    sub = df_ts[df_ts["Site"] == s].sort_values("Date_CS")
    if sub.empty:
        continue

    base_col = SITE_COLORS.get(s, "0.2")

    ax.plot(sub["Date_CS"], sub["Over_ha"],   "-o", ms=5, color=base_col)
    ax.plot(sub["Date_CS"], sub["Under_ha"], "--s", ms=5, color=base_col)
    ax.plot(sub["Date_CS"], sub["Inter_ha"], ":^", ms=5, color=base_col)

    stats = single_site_stats.get(s)
    u_date = uav_dates.get(s)
    title_extra = f" (IoU = {stats['IoU_pct']:.0f}%)" if stats is not None else ""

    if stats is not None and u_date is not None:
        r = stats["IoU_pct"] / 100.0
        O = stats["Over_ha"]
        U = stats["Under_ha"]
        I = (r * (O + U)) / (1.0 - r) if r < 1.0 else 0.0
        ax.plot(u_date, O, marker="*", ms=9, ls="None", color=base_col)
        ax.plot(u_date, U, marker="*", ms=9, ls="None", color=base_col)
        ax.plot(u_date, I, marker="*", ms=9, ls="None", color=base_col)

    ax.set_title(f"{s}{title_extra}", fontsize=12)

    if i == 1:
        ax.set_ylabel("Area (ha)", rotation=90, labelpad=10)
    else:
        ax.set_ylabel("")

    ax.xaxis.set_major_locator(YearLocator(base=1))
    ax.xaxis.set_major_formatter(DateFormatter("%Y"))

    if i < len(valid_sites) - 1:
        ax.tick_params(labelbottom=False)
    else:
        ax.set_xlabel("Time")
        fig.autofmt_xdate(rotation=45)

    ax.set_ylim(0.0, global_ylim)
    ax.set_yticks(yticks)
    ax.grid(True, linestyle=":", linewidth=0.5, alpha=0.3)

    panel_letter = chr(ord("B") + i)  # B, C, D
    ax.text(
        0.02, 0.98, panel_letter,
        transform=ax.transAxes,
        ha="left", va="top",
        fontsize=14, fontweight="bold",
    )

# -----------------------------------------------------------------------
# 10. Legend INSIDE panel A (bottom-left) + save
# -----------------------------------------------------------------------
ax_box.legend(
    handles=legend_handles_all,
    loc="lower left",
    bbox_to_anchor=(0.02, 0.02),
    frameon=False,
    ncol=1,
    borderpad=0.2,
    labelspacing=0.25,
    handletextpad=0.6,
)

os.makedirs(os.path.dirname(out_png_box), exist_ok=True)
plt.savefig(out_png_box)   # no tight_layout, constrained_layout handles spacing
plt.close(fig)

print(f"Saved combined IoU boxplot + time-series figure: {out_png_box}")
